In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def compute_glcm(image_gray, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4]):
    glcm = np.zeros((256, 256, len(distances), len(angles)), dtype=np.float64)
    for i, d in enumerate(distances):
        for j, theta in enumerate(angles):
            # Compute co-occurrence matrix
            co_occurrence = np.zeros((256, 256), dtype=np.uint8)
            for y in range(image_gray.shape[0]):
                for x in range(image_gray.shape[1]):
                    # Calculate the neighbor pixel coordinates
                    neighbor_y = y + int(d * np.sin(theta))
                    neighbor_x = x + int(d * np.cos(theta))
                    # Ensure neighbor is within image boundaries
                    if 0 <= neighbor_y < image_gray.shape[0] and 0 <= neighbor_x < image_gray.shape[1]:
                        intensity_pair = (image_gray[y, x], image_gray[neighbor_y, neighbor_x])
                        co_occurrence[intensity_pair] += 1
            glcm[:, :, i, j] = co_occurrence

    glcm_sum = np.sum(glcm, axis=(0, 1, 2, 3), keepdims=True)
    glcm /= glcm_sum
    return glcm

def compute_homogeneity(glcm):
    homogeneity = 0
    for i in range(glcm.shape[0]):
        for j in range(glcm.shape[1]):
            homogeneity += glcm[i, j] / (1 + np.abs(i - j))
    return homogeneity

def compute_entropy(glcm):
    entropy = 0
    for i in range(glcm.shape[0]):
        for j in range(glcm.shape[1]):
            if glcm[i, j] > 0:
                entropy += -glcm[i, j] * np.log(glcm[i, j])
    return entropy

# Load images
image1 = cv2.imread('/content/coarse.jpeg', cv2.IMREAD_GRAYSCALE)
image2 = cv2.imread('/content/random.jpeg', cv2.IMREAD_GRAYSCALE)
image3 = cv2.imread('/content/smooth.jpeg', cv2.IMREAD_GRAYSCALE)

# Compute GLCM for each image
glcm1 = compute_glcm(image1)
glcm2 = compute_glcm(image2)
glcm3 = compute_glcm(image3)

# Compute Homogeneity and Entropy for each image
homogeneity1 = compute_homogeneity(glcm1)
entropy1 = compute_entropy(glcm1)

homogeneity2 = compute_homogeneity(glcm2)
entropy2 = compute_entropy(glcm2)

homogeneity3 = compute_homogeneity(glcm3)
entropy3 = compute_entropy(glcm3)

print("Image 1 - Homogeneity:", homogeneity1, "Entropy:", entropy1)
print("Image 2 - Homogeneity:", homogeneity2, "Entropy:", entropy2)
print("Image 3 - Homogeneity:", homogeneity3, "Entropy:", entropy3)

# Display images
plt.figure(figsize=(10, 4))

plt.subplot(131)
plt.imshow(image1, cmap='gray')
plt.title('Image 1')
plt.axis('off')

plt.subplot(132)
plt.imshow(image2, cmap='gray')
plt.title('Image 2')
plt.axis('off')

plt.subplot(133)
plt.imshow(image3, cmap='gray')
plt.title('Image 3')
plt.axis('off')

plt.show()


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def compute_glcm_features(img):
  """
  Computes the GLCM texture features for a given image.

  Args:
    img: A 2D numpy array representing the image.

  Returns:
    A dictionary containing the GLCM texture features:
      - contrast: The contrast of the image.
      - correlation: The correlation of the image.
      - homogeneity: The homogeneity of the image.
      - energy: The energy of the image.
  """

  # Ensure the image is in uint8 format.
  img = img.astype(np.uint8)

  # Calculate the co-occurrence matrix.
  cooccurrence_matrix = np.zeros((256, 256))
  for i in range(img.shape[0] - 1):
    for j in range(img.shape[1] - 1):
      cooccurrence_matrix[img[i, j], img[i + 1, j]] += 1

  # Calculate the GLCM texture features.
  contrast = 0.0
  correlation = 0.0
  homogeneity = 0.0
  energy = 0.0
  for i in range(256):
    for j in range(256):
      contrast += (i - j) ** 2 * cooccurrence_matrix[i, j]
      correlation += (i - img.mean()) * (j - img.mean()) * cooccurrence_matrix[i, j]
      homogeneity += cooccurrence_matrix[i, j] / (1 + abs(i - j))
      energy += cooccurrence_matrix[i, j] ** 2

  return {
    "contrast": contrast,
    "correlation": correlation,
    "homogeneity": homogeneity,
    "energy":energy}

# Load the images.
smooth_img = plt.imread('/content/smooth.jpeg')
random_img = plt.imread('/content/random.jpeg')
coarse_img = plt.imread('/content/coarse.jpeg')

# Compute the GLCM texture features.
smooth_features = compute_glcm_features(smooth_img)
random_features = compute_glcm_features(random_img)
coarse_features = compute_glcm_features(coarse_img)

# Print the results.
print('Smooth Image:')
print(smooth_features)
print('Random Image:')
print(random_features)
print('Coarse Image:')
print(coarse_features)


Smooth Image:
{'contrast': 205645.0, 'correlation': 16831068.82431552, 'homogeneity': 68985.57695082681, 'energy': 71027746.0}
Random Image:
{'contrast': 175891382.0, 'correlation': 168907925.56044772, 'homogeneity': 10586.575984279905, 'energy': 2982566.0}
Coarse Image:
{'contrast': 11223695.0, 'correlation': 83548784.86294281, 'homogeneity': 19798.46267997269, 'energy': 3422522.0}


In [ ]:
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt

def compute_glcm_features(img):
    """
    Computes the GLCM texture features for a given image.

    Args:
        img: A 2D numpy array representing the image.

    Returns:
        A dictionary containing the GLCM texture features:
            - contrast: The contrast of the image.
            - correlation: The correlation of the image.
            - homogeneity: The homogeneity of the image.
            - energy: The energy of the image.
    """

    # Ensure the image is in uint8 format.
    img = img.astype(np.uint8)

    # Calculate the co-occurrence matrix.
    cooccurrence_matrix = np.zeros((256, 256))
    for i in range(img.shape[0] - 1):
        for j in range(img.shape[1] - 1):
            cooccurrence_matrix[img[i, j], img[i + 1, j]] += 1

    # Calculate the GLCM texture features.
    contrast = 0.0
    correlation = 0.0
    homogeneity = 0.0
    energy = 0.0
    for i in range(256):
        for j in range(256):
            contrast += (i - j) ** 2 * cooccurrence_matrix[i, j]
            correlation += (i - img.mean()) * (j - img.mean()) * cooccurrence_matrix[i, j]
            homogeneity += cooccurrence_matrix[i, j] / (1 + abs(i - j))
            energy += cooccurrence_matrix[i, j] ** 2

    return {
        "Contrast": contrast,
        "Correlation": correlation,
        "Homogeneity": homogeneity,
        "Energy": energy
    }

# Load the images.
smooth_img = plt.imread('/content/smooth.jpeg')
random_img = plt.imread('/content/random.jpeg')
coarse_img = plt.imread('/content/coarse.jpeg')

# Compute the GLCM texture features.
smooth_features = compute_glcm_features(smooth_img)
random_features = compute_glcm_features(random_img)
coarse_features = compute_glcm_features(coarse_img)

# Organize data into tabular format
data = [
    ['Smooth Image'] + [smooth_features[key] for key in smooth_features],
    ['Random Image'] + [random_features[key] for key in random_features],
    ['Coarse Image'] + [coarse_features[key] for key in coarse_features]
]

# Define headers
headers = ['Image', 'Contrast', 'Correlation', 'Homogeneity', 'Energy']

# Print the results in tabular form
print(tabulate(data, headers=headers, tablefmt='grid'))


+--------------+------------------+---------------+---------------+-------------+
| Image        |         Contrast |   Correlation |   Homogeneity |      Energy |
+==============+==================+===============+===============+=============+
| Smooth Image | 205645           |   1.68311e+07 |       68985.6 | 7.10277e+07 |
+--------------+------------------+---------------+---------------+-------------+
| Random Image |      1.75891e+08 |   1.68908e+08 |       10586.6 | 2.98257e+06 |
+--------------+------------------+---------------+---------------+-------------+
| Coarse Image |      1.12237e+07 |   8.35488e+07 |       19798.5 | 3.42252e+06 |
+--------------+------------------+---------------+---------------+-------------+
